# Testing homemade layers

This notebook contains tests for all the homemade layers. The tests takes some samples from the mni-
st dataset as a minibatch and compares the results from running this minibatch through the test CNN
to running the same minibatch through the homemade layers.

In [1]:
#importing relevant libraries

import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, TensorDataset, Subset
import torch.optim as optim
import torch.nn.functional as F
import os

from CNN_small_architecture import CNNSmall
from CNN_layers import conv_homemade
from CNN_layers import maxpool_homemade
from CNN_layers import batchnorm_homemade
from CNN_layers import linear_layer_homemade

In [2]:
def tokenize(num):
    if num == 1:
        return torch.tensor(np.array([1., 0.]))
    else:
        return torch.tensor(np.array([0., 1.]))

MNIST_test = datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
test_set = [[data[0], tokenize(data[1])] for data in MNIST_test if data[1] in [1,2]]

batch_size = 2
test_loader = DataLoader(test_set, batch_size=batch_size)

In [3]:
# takes a 4-dimensional tensor from the dataloader (batch_size x channel_size x height x width)
# and turns it into a list of lists of (height x width) numpy arrays
def transform_input(input_batch):
    return list(list(input_batch.detach().numpy()))

# compares two lists of batches with same amount of channels 
# and returns the summed absoute loss for each batch over the channels
def compare(x, y):
    b_diff = []
    batch_size = len(x)

    for b in range(batch_size):
        c_diff = []
        channels = len(x[b])

        for c in range(channels):
            diff = np.sum(np.absolute(x[b][c] - y[b][c]))
            c_diff.append(diff)
        
        b_diff.append(c_diff)
    
    return b_diff

In [4]:
model_original = CNNSmall()
model_original.eval()

path = "CNN_small"
load = True

if load and os.path.isfile(path):
    model_original.load_state_dict(torch.load(path))

In [5]:
input_original_test, label_test = next(iter(test_loader))
print(np.array(input_original_test).shape)
input_homemade_test = transform_input(input_batch=input_original_test)

(2, 1, 28, 28)


In [6]:
def create_conv_homemade(model_conv):
    weights = model_conv.weight
    biases = model_conv.bias.detach().numpy()

    out_c, in_c, r, c = weights.shape
    conv1_filters = []

    for f in range(out_c):
        filter_ = []
        kernels = []

        for kernel in list(weights[f,:,:,:]):
            kernels.append(kernel.detach().numpy())

        filter_.append(kernels)
        filter_.append(biases[f])
        conv1_filters.append(filter_)
    
    return conv_homemade.Conv(filters=conv1_filters, in_channels=in_c)


In [7]:
def create_batchnorm_homemade(model_batchnorm):
    weights = model_batchnorm.weight
    biases = model_batchnorm.bias
    running_mean = model_batchnorm.running_mean
    running_var = model_batchnorm.running_var

    return batchnorm_homemade.BatchNorm(weights=weights, biases=biases, running_mean = running_mean, running_var = running_var)

In [8]:
def create_maxpool_homemade(model_maxpool):
    kernel_size = model_maxpool.kernel_size
    stride = model_maxpool.stride
    if type(model_maxpool.padding) == int:
        padding = (model_maxpool.padding, model_maxpool.padding)
    else:
        padding = model_maxpool.padding
    
    return maxpool_homemade.MaxPool(kernel_size=kernel_size, stride=stride, padding=padding)
    


In [9]:
conv1_homemade = create_conv_homemade(model_conv= model_original.conv1)
batchnorm1_homemade = create_batchnorm_homemade(model_batchnorm= model_original.batchNorm1)
maxpool1_homemade = create_maxpool_homemade(model_maxpool= model_original.maxPool1)
conv2_homemade = create_conv_homemade(model_conv= model_original.conv2)
batchnorm2_homemade = create_batchnorm_homemade(model_batchnorm= model_original.batchNorm2)
maxpool2_homemade = create_maxpool_homemade(model_original.maxPool2)
linear_homemade = linear_layer_homemade.linear_layer(model_original.lin.weight,model_original.lin.bias,2)

STRIDE ER PRÆDEFINERET
STRIDE ER PRÆDEFINERET


## Testing the first convolutional layer

In [10]:
# homemade conv1 filter on test
out_homemade = conv1_homemade(input_homemade_test)
# original conv1 filter on test
out_original = model_original.conv1(input_original_test)

print("Convolutional layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

Convolutional layer 1 error over out channels: 


[[2.4508228573666413e-06, 2.7754352047160147e-06, 2.4542343283751045e-06],
 [3.954176872073001e-06, 5.6260658496676275e-06, 4.12328020808761e-06]]

## Testing the first Batchnorm layer

In [11]:
# homemade batchNorm1 filter on test
out_homemade = batchnorm1_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.batchNorm1(out_original)

print("BatchNorm layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

BatchNorm layer 1 error over out channels: 


[[5.3554773330688477e-05, 4.4269487261772156e-05, 5.099223926663399e-05],
 [6.334856152534485e-05, 7.360382005572319e-05, 6.171129643917084e-05]]

## Testing the first Maxpool layer

In [12]:
# homemade MaxPool filter on test
out_homemade = maxpool1_homemade(input_batch=out_homemade)
# original MaxPool filter on test
out_original = model_original.maxPool1(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

MaxPool layer 1 error over out channels: 


[[1.4118850231170654e-05, 1.1965632438659668e-05, 1.4308840036392212e-05],
 [1.714937388896942e-05, 2.1882355213165283e-05, 1.6798265278339386e-05]]

## Testing the second Convolutional layer

In [13]:
# TODO: test the second convolutional layer

# homemade batchNorm1 filter on test
out_homemade = conv2_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.conv2(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

MaxPool layer 1 error over out channels: 


[[6.839803987607196e-06,
  1.13506730272106e-05,
  8.365254015374468e-06,
  1.1760292614348966e-05,
  8.43778021558661e-06],
 [1.4010783349727163e-05,
  1.4851634311224826e-05,
  1.2323523619323096e-05,
  1.5419261641030912e-05,
  1.422003913834069e-05]]

## Testing the second Batchnorm layer

In [14]:
# homemade batchNorm1 filter on test
out_homemade = batchnorm2_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.batchNorm2(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

MaxPool layer 1 error over out channels: 


[[1.0855495929718018e-05,
  2.0815059542655945e-05,
  1.3127923011779785e-05,
  1.8320977687835693e-05,
  1.2755393981933594e-05],
 [2.2135674953460693e-05,
  2.8287991881370544e-05,
  1.8671154975891113e-05,
  2.382509410381317e-05,
  2.467632293701172e-05]]

## Testing the second Maxpool layer

In [15]:
# homemade batchNorm1 filter on test
out_homemade = maxpool2_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.maxPool2(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

MaxPool layer 1 error over out channels: 


[[2.0116567611694336e-06,
  3.7476420402526855e-06,
  1.8402934074401855e-06,
  3.3676624298095703e-06,
  1.8551945686340332e-06],
 [4.991888999938965e-06,
  4.827976226806641e-06,
  1.1920928955078125e-06,
  4.32133674621582e-06,
  2.9206275939941406e-06]]

## Testing the Linear layer

In [16]:
# homemade linear filter on test
out_homemade = linear_homemade(torch.reshape(torch.tensor(out_homemade, dtype = torch.double),(2,45)))
# original linear filter on test
out_original = model_original.lin(torch.reshape(out_original, (2,45)))

print("Linear layer 1 error over out channels: ")
compare(out_homemade.numpy(), list(list(out_original.detach().numpy())))

tmp: tensor(3.8711, dtype=torch.float64)
tensor(0.1424, dtype=torch.float64)
tensor(4.0135, dtype=torch.float64)
tmp: tensor(-4.5000, dtype=torch.float64)
tensor(0.1424, dtype=torch.float64)
tensor(-4.3576, dtype=torch.float64)
tensor([[ 4.0135, -3.6916],
        [-4.3576,  6.3461]], dtype=torch.float64)
tensor([[ 4.0135, -3.6916],
        [-4.3576,  6.3461]], grad_fn=<AddmmBackward0>)
Linear layer 1 error over out channels: 


C:\Users\sture\AppData\Local\Temp\ipykernel_1688\2926538855.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\b\abs_bao0hdcrdh\croot\pytorch_1675190257512\work\torch\csrc\utils\tensor_new.cpp:204.)
  out_homemade = linear_homemade(torch.reshape(torch.tensor(out_homemade, dtype = torch.double),(2,45)))
c:\Users\sture\Documents\Studie\Bachelor\bachelorprojekt\CNN_layers\linear_layer_homemade.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tmp = torch.dot(input[n],torch.tensor(self.weights[i], dtype = torch.double)) + torch.tensor(self.bias[i], dtype = torch.double)
c:\Users\sture\Documents\Studie\Bachelor\bachelorprojekt\CNN_layers\linear_layer_homemade.py:20:

[[2.3730480247508012e-07, 4.3005196914691624e-07],
 [2.7937032829328245e-06, 1.0375646679605666e-06]]

In [17]:
from CNN_layers.linear_layer import linear_layer

ModuleNotFoundError: No module named 'CNN_layers.linear_layer'

tensor([ 0.0055, -0.3833, -0.0223, -0.1087, -0.0395, -0.1584, -0.2102, -0.3390,
        -0.1260,  0.1047,  0.0076,  0.3647, -0.0429, -0.3491,  0.1892, -0.1851,
        -0.1496, -0.0308,  0.3081,  0.1367, -0.1633,  0.2652, -0.1780,  0.0351,
         0.4791, -0.0754,  0.1078, -0.1172,  0.3277,  0.2573, -0.0378, -0.2831,
         0.0229,  0.2404,  0.1599, -0.3007, -0.1929, -0.2463, -0.3025,  0.2377,
         0.1619, -0.2581, -0.0129, -0.2355, -0.3202],
       grad_fn=<SelectBackward0>)
tensor([ 0.0055, -0.3833, -0.0223, -0.1087, -0.0395, -0.1584, -0.2102, -0.3390,
        -0.1260,  0.1047,  0.0076,  0.3647, -0.0429, -0.3491,  0.1892, -0.1851,
        -0.1496, -0.0308,  0.3081,  0.1367, -0.1633,  0.2652, -0.1780,  0.0351,
         0.4791, -0.0754,  0.1078, -0.1172,  0.3277,  0.2573, -0.0378, -0.2831,
         0.0229,  0.2404,  0.1599, -0.3007, -0.1929, -0.2463, -0.3025,  0.2377,
         0.1619, -0.2581, -0.0129, -0.2355, -0.3202], grad_fn=<MulBackward0>)


In [ ]:
lin = linear_layer(model_original.lin.weight,model_original.lin.bias,2)

lin_in = 

out = lin(torch.flatten(torch.tensor(out_homemade)))

SyntaxError: invalid syntax (2866144332.py, line 3)